In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [2]:
SERVER = 'DESKTOP-LQOJV4Q\SQLEXPRESS'
DATABASE = 'shamansk_sandbox'
DRIVER = 'ODBC Driver 17 for SQL Server'
DATABASE_CONNECTION = f'mssql://@{SERVER}/{DATABASE}?driver={DRIVER}'
engine = create_engine(DATABASE_CONNECTION)

In [34]:
#conn = engine.connect()
#df = pd.read_sql_query("select * from [dbo].[sample_data]", conn) 
query_sales = '''
        SELECT date_ymd, product_name, category, SUM(revenue) revenue, SUM(sale_quantity) sale_quantity, customer_name
        FROM sales s
        INNER JOIN product p ON s.product_id = p.product_id
        LEFT JOIN customer c ON s.customer_id = c.customer_id
        WHERE YEAR(date_ymd) in (2017, 2018, 2019)
        Group by date_ymd, product_name, category, customer_name
    '''
df_query_sales = pd.read_sql(query_sales, engine)
df_query_sales.head()


,date_ymd,product_name,category,revenue,sale_quantity,customer_name
0,2017-01-01,товар 10018,категория 99,980.00,44319.0,клиент 1004
1,2017-01-01,товар 10047,категория 99,1550.00,44318.0,клиент 1013
2,2017-01-01,товар 10060,категория 99,952.75,31048.0,клиент 1002
3,2017-01-01,товар 10102,категория 99,-19320.00,-64.4,клиент 1025
4,2017-01-01,товар 10103,категория 99,14573.00,76.7,клиент 1006


In [35]:
df_query_sales.describe(include='all')

,date_ymd,product_name,category,revenue,sale_quantity,customer_name
count,63382,63382,63382,63375.000000,63382.000000,63382
unique,36,3959,9,NaN,NaN,742
top,2019-09-01,товар 15344,категория 03,NaN,NaN,клиент 1289
freq,2805,1257,24505,NaN,NaN,3188
mean,NaN,NaN,NaN,6714.286346,1193.595643,NaN
std,NaN,NaN,NaN,18229.122995,7092.022821,NaN
min,NaN,NaN,NaN,-259610.000000,-1200.000000,NaN
25%,NaN,NaN,NaN,804.000000,2.000000,NaN
50%,NaN,NaN,NaN,2360.000000,5.000000,NaN
75%,NaN,NaN,NaN,6700.000000,20.000000,NaN


In [5]:
df_query_sales.isnull().sum()

date_ymd         0
product_name     0
category         0
revenue          7
sale_quantity    0
customer_name    0
dtype: int64

In [36]:
df_query_sales_cat = df_query_sales.groupby('category')['revenue'].sum().sort_values(ascending = False)
df_query_sales_cat

category
категория 03    2.414636e+08
категория 05    7.853291e+07
категория 99    3.469779e+07
категория 01    2.051600e+07
категория 04    1.811326e+07
категория 02    1.442481e+07
категория 07    1.390318e+07
категория 06    3.847692e+06
категория 66    1.860300e+04
Name: revenue, dtype: float64

In [39]:
# ограничиваем строки категорией 03
df = df_query_sales.loc[df_query_sales.loc[:,'category'] == 'категория 03', :]
# ограничиваем столбцы до двух, группируем
df = df.loc[:, ['date_ymd', 'sale_quantity']] \
.groupby('date_ymd')['sale_quantity'].sum()
df

date_ymd
2017-01-01    2013.0
2017-02-01    2731.0
2017-03-01    3303.0
2017-04-01    3094.0
2017-05-01    3008.0
2017-06-01    2833.0
2017-07-01    2782.0
2017-08-01    3649.0
2017-09-01    3418.0
2017-10-01    3704.0
2017-11-01    2994.0
2017-12-01    3965.0
2018-01-01    2729.0
2018-02-01    3389.0
2018-03-01    3557.0
2018-04-01    3009.0
2018-05-01    3221.0
2018-06-01    4328.0
2018-07-01    4035.0
2018-08-01    4580.0
2018-09-01    4653.0
2018-10-01    4155.0
2018-11-01    4224.0
2018-12-01    5552.0
2019-01-01    2420.0
2019-02-01    3590.0
2019-03-01    4015.0
2019-04-01    3868.0
2019-05-01    4143.0
2019-06-01    3735.0
2019-07-01    4820.0
2019-08-01    4970.0
2019-09-01    4228.0
2019-10-01    4655.0
2019-11-01    4012.0
2019-12-01    4856.0
Name: sale_quantity, dtype: float64

In [40]:
# преобразуем в DataFrame
df = df.to_frame('sale_quantity').reset_index()
df

,date_ymd,sale_quantity
0,2017-01-01,2013.0
1,2017-02-01,2731.0
2,2017-03-01,3303.0
3,2017-04-01,3094.0
4,2017-05-01,3008.0
5,2017-06-01,2833.0
6,2017-07-01,2782.0
7,2017-08-01,3649.0
8,2017-09-01,3418.0
9,2017-10-01,3704.0


In [57]:
cat = 'категория 03'


df = df_query_sales.query('category == "категория 03"') \
.groupby(['category', 'date_ymd'], as_index=False) \
.agg({'sale_quantity':sum})[['date_ymd', 'sale_quantity']]

         

         
df

,date_ymd,sale_quantity
0,2017-01-01,2013.0
1,2017-02-01,2731.0
2,2017-03-01,3303.0
3,2017-04-01,3094.0
4,2017-05-01,3008.0
5,2017-06-01,2833.0
6,2017-07-01,2782.0
7,2017-08-01,3649.0
8,2017-09-01,3418.0
9,2017-10-01,3704.0
